# Modélisation #3 XGBoost + Optuna

In [12]:
# Importation des packages

# Base
import importlib
import functions
importlib.reload(functions)
from functions import *

# Data Management
import pandas as pd
import numpy as np

# Modélisation
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Optimisation
import optuna 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
# Importation des données

train_data = pd.read_csv('../data/Train_Data/3_train_data.csv', index_col=0)
train_scores = pd.read_csv('../data/Y_train_1rknArQ.csv', index_col=0)

sub_data = pd.read_csv('../data/Sub_Data/3_sub_data.csv', index_col=0)

train_data.head(5)

,HOME_TEAM_SHOTS_TOTAL_season_sum,HOME_TEAM_SHOTS_INSIDEBOX_season_sum,HOME_TEAM_SHOTS_OFF_TARGET_season_sum,HOME_TEAM_SHOTS_ON_TARGET_season_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_season_sum,HOME_TEAM_PASSES_season_sum,HOME_TEAM_SUCCESSFUL_PASSES_season_sum,HOME_TEAM_SAVES_season_sum,HOME_TEAM_CORNERS_season_sum,HOME_TEAM_FOULS_season_sum,HOME_TEAM_YELLOWCARDS_season_sum,HOME_TEAM_REDCARDS_season_sum,HOME_TEAM_OFFSIDES_season_sum,HOME_TEAM_ATTACKS_season_sum,HOME_TEAM_PENALTIES_season_sum,HOME_TEAM_SUBSTITUTIONS_season_sum,HOME_TEAM_BALL_SAFE_season_sum,HOME_TEAM_DANGEROUS_ATTACKS_season_sum,HOME_TEAM_INJURIES_season_sum,HOME_TEAM_GOALS_season_sum,HOME_TEAM_GAME_WON_season_sum,HOME_TEAM_GAME_DRAW_season_sum,HOME_TEAM_GAME_LOST_season_sum,HOME_TEAM_SHOTS_TOTAL_season_average,HOME_TEAM_SHOTS_INSIDEBOX_season_average,HOME_TEAM_SHOTS_OFF_TARGET_season_average,HOME_TEAM_SHOTS_ON_TARGET_season_average,HOME_TEAM_SHOTS_OUTSIDEBOX_season_average,HOME_TEAM_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_season_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_average,HOME_TEAM_SAVES_season_average,HOME_TEAM_CORNERS_season_average,HOME_TEAM_BALL_POSSESSION_season_average,HOME_TEAM_FOULS_season_average,HOME_TEAM_YELLOWCARDS_season_average,HOME_TEAM_REDCARDS_season_average,HOME_TEAM_OFFSIDES_season_average,HOME_TEAM_ATTACKS_season_average,HOME_TEAM_PENALTIES_season_average,HOME_TEAM_SUBSTITUTIONS_season_average,HOME_TEAM_BALL_SAFE_season_average,HOME_TEAM_DANGEROUS_ATTACKS_season_average,HOME_TEAM_INJURIES_season_average,HOME_TEAM_GOALS_season_average,HOME_TEAM_GAME_WON_season_average,HOME_TEAM_GAME_DRAW_season_average,HOME_TEAM_GAME_LOST_season_average,HOME_TEAM_SHOTS_TOTAL_season_std,HOME_TEAM_SHOTS_INSIDEBOX_season_std,HOME_TEAM_SHOTS_OFF_TARGET_season_std,HOME_TEAM_SHOTS_ON_TARGET_season_std,HOME_TEAM_SHOTS_OUTSIDEBOX_season_std,HOME_TEAM_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_season_std,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std,HOME_TEAM_SAVES_season_std,HOME_TEAM_CORNERS_season_std,HOME_TEAM_BALL_POSSESSION_season_std,HOME_TEAM_FOULS_season_std,HOME_TEAM_YELLOWCARDS_season_std,HOME_TEAM_REDCARDS_season_std,HOME_TEAM_OFFSIDES_season_std,HOME_TEAM_ATTACKS_season_std,HOME_TEAM_PENALTIES_season_std,HOME_TEAM_SUBSTITUTIONS_season_std,HOME_TEAM_BALL_SAFE_season_std,HOME_TEAM_DANGEROUS_ATTACKS_season_std,HOME_TEAM_INJURIES_season_std,HOME_TEAM_GOALS_season_std,HOME_TEAM_SHOTS_TOTAL_5_last_match_sum,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_sum,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_sum,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_sum,HOME_TEAM_PASSES_5_last_match_sum,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_sum,HOME_TEAM_SAVES_5_last_match_sum,HOME_TEAM_CORNERS_5_last_match_sum,HOME_TEAM_FOULS_5_last_match_sum,HOME_TEAM_YELLOWCARDS_5_last_match_sum,HOME_TEAM_REDCARDS_5_last_match_sum,HOME_TEAM_OFFSIDES_5_last_match_sum,HOME_TEAM_ATTACKS_5_last_match_sum,HOME_TEAM_PENALTIES_5_last_match_sum,HOME_TEAM_SUBSTITUTIONS_5_last_match_sum,HOME_TEAM_BALL_SAFE_5_last_match_sum,HOME_TEAM_DANGEROUS_ATTACKS_5_last_match_sum,HOME_TEAM_INJURIES_5_last_match_sum,HOME_TEAM_GOALS_5_last_match_sum,HOME_TEAM_GAME_WON_5_last_match_sum,HOME_TEAM_GAME_DRAW_5_last_match_sum,HOME_TEAM_GAME_LOST_5_last_match_sum,HOME_TEAM_SHOTS_TOTAL_5_last_match_average,HOME_TEAM_SHOTS_INSIDEBOX_5_last_match_average,HOME_TEAM_SHOTS_OFF_TARGET_5_last_match_average,HOME_TEAM_SHOTS_ON_TARGET_5_last_match_average,HOME_TEAM_SHOTS_OUTSIDEBOX_5_last_match_average,HOME_TEAM_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_5_last_match_average,HOME_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_5_last_match_average,HOME_TEAM_SAVES_5_last_match_average,HOME_TEAM_CORNERS_5_last_match_average,HOME_TEAM_BALL_POSSESSION_5_last_match_average,HOME_TEAM_FOULS_5_last_match_average,HOME_TEAM_YELLOWCARDS_5_last_match_average,HOME_TEAM_REDCARDS_5_last_match_average,HOME_TEAM_OFFSIDES_5_last_match_average,HOME_TEAM_ATTACKS_5_last_match_average,HOME_TEAM_PENALTI

In [14]:
# Je préfère être sûr que les indexs concordent. 

train_scores = train_scores.loc[train_data.index]

train_scores.head(5)

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
0,0,0,1
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0


In [15]:
# Préparation des données pour la prédiction. 

train_scores_1c = train_scores[['HOME_WINS', 'DRAW', 'AWAY_WINS']].idxmax(axis=1)
label_mapping = {'HOME_WINS': 0, 'DRAW': 1, 'AWAY_WINS': 2}
train_scores_1c = train_scores_1c.replace(label_mapping)

train_scores_1c.head(5)

/var/folders/5w/37p1__596xz7ylx6ltmmggph0000gn/T/ipykernel_7582/2909335398.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_scores_1c = train_scores_1c.replace(label_mapping)


ID
0    2
1    1
2    2
3    0
4    1
dtype: int64

In [16]:
x_train, x_test, y_train, y_test = train_test_split(train_data, train_scores_1c, train_size=0.8, random_state=42)

In [17]:
def objective(trial):
    # Définit les hyperparamètres que l'on souhaite optimiser
    param = {
        'objective': 'multi:softprob',  # Pour la classification multi-classes
        'num_class': 3,  # Nombre de classes dans les données
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 1.0),  # régularisation L2
        'alpha': trial.suggest_float('alpha', 1e-3, 1.0)  # régularisation L1
    }

    # Entraînement du modèle avec les hyperparamètres choisis
    model = xgb.XGBClassifier(**param, eval_metric='mlogloss')
    model.fit(x_train, y_train)

    # Prédiction
    preds = model.predict(x_test)
    
    # Calculer l'accuracy
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy

study = optuna.create_study(direction="maximize")  # Maximise l'accuracy
study.optimize(objective, n_trials=1000)  # Nombre d'essais pour la recherche

print("Meilleurs hyperparamètres trouvés :", study.best_params)
print("Meilleure accuracy :", study.best_value)

[I 2024-09-20 12:52:45,479] A new study created in memory with name: no-name-25ec89e1-97ba-4f57-8b34-a7b1243d95f0
[I 2024-09-20 12:53:05,270] Trial 0 finished with value: 0.4928890694839496 and parameters: {'max_depth': 5, 'learning_rate': 0.01139332343806157, 'n_estimators': 912, 'min_child_weight': 10, 'gamma': 0.17264773006618228, 'subsample': 0.802720331966071, 'colsample_bytree': 0.5497978529544295, 'lambda': 0.47027227837418356, 'alpha': 0.5923611470974807}. Best is trial 0 with value: 0.4928890694839496.
[I 2024-09-20 12:53:56,956] Trial 1 finished with value: 0.4900446972775295 and parameters: {'max_depth': 9, 'learning_rate': 0.013512980070490573, 'n_estimators': 745, 'min_child_weight': 8, 'gamma': 0.2706967704577865, 'subsample': 0.8626700426534497, 'colsample_bytree': 0.9883255542104785, 'lambda': 0.7032372138503429, 'alpha': 0.4178082915781405}. Best is trial 0 with value: 0.4928890694839496.
[I 2024-09-20 12:54:29,383] Trial 2 finished with value: 0.47338480292564 and par

Meilleurs hyperparamètres trouvés : {'max_depth': 3, 'learning_rate': 0.02714315173477086, 'n_estimators': 360, 'min_child_weight': 2, 'gamma': 0.12267570709279488, 'subsample': 0.7438009478466107, 'colsample_bytree': 0.580864166397642, 'lambda': 0.34939874832404394, 'alpha': 0.21916954742499636}
Meilleure accuracy : 0.5095489638358391


In [24]:
# Reformation du modèle

# best_params = study.best_params
# best_model = xgb.XGBClassifier(**best_params)

best_model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=3,
    max_depth=3,
    learning_rate=0.02714315173477086,
    n_estimators=360,
    min_child_weight= 2,
    gamma= 0.12267570709279488,
    subsample= 0.7438009478466107,
    colsample_bytree=0.580864166397642,
    reg_lambda=0.34939874832404394,
    alpha=0.21916954742499636
)

best_model.fit(x_train, y_train)

y_pred = best_model.predict(x_test)

In [26]:
final_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy finale : {final_accuracy}")

Accuracy finale : 0.5095489638358391


# Prédictions

In [27]:
y_sub = best_model.predict(sub_data)

In [28]:
# Reconstruction du DataFrame y_sub avec les colonnes 'HOME_WINS', 'DRAW', 'AWAY_WINS'
y_sub_df = pd.DataFrame(y_sub, columns=['PRED'])

# Transformation en one-hot encoding pour obtenir les 3 colonnes
y_sub_df['HOME_WINS'] = (y_sub_df['PRED'] == 0).astype(int)
y_sub_df['DRAW'] = (y_sub_df['PRED'] == 1).astype(int)
y_sub_df['AWAY_WINS'] = (y_sub_df['PRED'] == 2).astype(int)

# Ajout de la colonne 'ID' depuis sub_data
y_sub_df['ID'] = sub_data.index

# Suppression de la colonne 'PRED'
y_sub_df.drop('PRED', axis=1, inplace=True)

# Réorganiser les colonnes pour avoir 'ID' en premier
y_sub_df = y_sub_df[['ID', 'HOME_WINS', 'DRAW', 'AWAY_WINS']]

# Afficher les premières lignes du résultat final
y_sub_df = y_sub_df.set_index('ID')
y_sub_df.head()

,HOME_WINS,DRAW,AWAY_WINS
ID,,,
12303.0,0,0,1
12304.0,0,0,1
12305.0,1,0,0
12306.0,1,0,0
12307.0,0,0,1


In [30]:
y_sub_df.to_csv('../data/Sub_Data/7_sub_scores.csv')